# Pigpen PyTorch Example
by Joe Norton

In [24]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [25]:
import random
from PigpenDataset import PigpenDatasetClass


In [26]:
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.5,), (0.5,))  # Example normalization
])

train_dataset = PigpenDatasetClass('train', transform=transform)
test_dataset = PigpenDatasetClass('test', transform=transform)
valid_dataset = PigpenDatasetClass('validation', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=28, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=28, shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=28, shuffle=False)

In [27]:
len(train_dataset), len(test_dataset), len(valid_dataset)

(53974, 13500, 13500)

In [28]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 28)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=28, bias=True)
  )
)


In [29]:

# Create the network, define a loss function, and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [30]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [31]:
epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(valid_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3.355399  [   28/53974]


loss: 2.950927  [ 2828/53974]
loss: 1.358134  [ 5628/53974]
loss: 0.877169  [ 8428/53974]
loss: 0.493135  [11228/53974]
loss: 0.355321  [14028/53974]
loss: 0.493009  [16828/53974]
loss: 0.494101  [19628/53974]
loss: 0.130580  [22428/53974]
loss: 0.090367  [25228/53974]
loss: 0.190067  [28028/53974]
loss: 0.721036  [30828/53974]
loss: 0.426996  [33628/53974]
loss: 0.100826  [36428/53974]
loss: 0.213606  [39228/53974]
loss: 0.175190  [42028/53974]
loss: 0.081666  [44828/53974]
loss: 0.102770  [47628/53974]
loss: 0.119312  [50428/53974]
loss: 0.039922  [53228/53974]
Test Error: 
 Accuracy: 46.0%, Avg loss: 3.173775 

Epoch 2
-------------------------------
loss: 0.217161  [   28/53974]
loss: 0.054290  [ 2828/53974]
loss: 0.046774  [ 5628/53974]
loss: 0.362513  [ 8428/53974]
loss: 0.052166  [11228/53974]
loss: 0.035143  [14028/53974]
loss: 0.040843  [16828/53974]
loss: 0.011575  [19628/53974]
loss: 0.058826  [22428/53974]
loss: 0.092925  [25228/53974]
loss: 0.026471  [28028/53974]
loss: 0.

In [32]:
test(test_loader, model, loss_fn)

Test Error: 
 Accuracy: 44.3%, Avg loss: 3.622199 

